In [ ]:
#default_exp web.db

# Web Db

> Handles sqlite DB access and initialization.

In [ ]:
#export
import click, sqlite3
from flask import current_app, g
from flask.cli import with_appcontext

In [ ]:
#export
def get_db():
    """Connect to the application's configured database. The connection
    is unique for each request and will be reused if this is called
    again.
    """
    if "db" not in g:
        g.db = sqlite3.connect(
            current_app.config["DATABASE"], detect_types=sqlite3.PARSE_DECLTYPES
        )
        g.db.row_factory = sqlite3.Row
    return g.db

def close_db(exception=None):
    """If this request connected to the database, close the
    connection.
    """
    db = g.pop("db", None)
    if db is not None: db.close()

def init_db():
    """Clear existing data and create new tables."""
    db = get_db()
    with current_app.open_resource("schema.sql") as f:
        db.executescript(f.read().decode("utf8"))

@click.command("init-db")
@with_appcontext
def init_db_command():
    """Clear existing data and create new tables."""
    init_db()
    click.echo("Initialized the database.")
    
def init_app(app):
    """Register database functions with the Flask app. This is called by
    the application factory.
    """
    app.teardown_appcontext(close_db)
    app.cli.add_command(init_db_command)

In [ ]:
#hide
from flask import Flask
from contextlib import contextmanager
from pathlib import Path
import tempfile, shutil, os, web_journal.web

def cd(p):
    print('Changing working directory:') # TODO: maybe we want to suppress stdout
    get_ipython().run_line_magic('cd', str(p))

@contextmanager
def test_resources():
    db_fd, db_path = tempfile.mkstemp()
    try:
        cwd = get_ipython().run_line_magic('pwd', '')
        cd(Path(web_journal.web.__file__).parent)
        app = Flask(__name__, instance_relative_config=True)
        app.config.from_mapping(SECRET_KEY="dev",TESTING=True,DATABASE=db_path)
        init_app(app)
        with app.app_context():
            yield app
    finally:
        cd(cwd)
        os.close(db_fd)
        os.unlink(db_path) # this would fail if close_db has not been called

In [ ]:
with test_resources() as app:
    db = get_db()
    assert db == get_db()
    cursor = db.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    assert [] == cursor.fetchall()
    init_db()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND NAME NOT LIKE 'sql%'")
    assert ['post','user'] == sorted([r[0] for r in cursor.fetchall()])

Changing working directory:
C:\Users\Butterp\github\pete88b\web_journal\web_journal\web
Changing working directory:
C:\Users\Butterp\github\pete88b\web_journal
